In [25]:
import pandas as pd
from pymongo import MongoClient
import json
from bson import json_util


In [3]:
viajes = pd.read_csv('viajes.csv')

In [4]:
viajes

,Unnamed: 0,codigo_viaje,tipo_de_gasto,gasto_observable,monto,inicio,fin,Duración del viaje,destino_mpio,destino_edo,dirección,unidad
0,0,V00001,Viáticos f,x,190.000000,25/01/2023,26/01/2023,2,León,GTO,DG1,Dirección general
1,1,V00002,Viáticos f,x,171.000000,26/01/2023,27/01/2023,2,Morelia,MCH,DG1,Dirección general
2,2,V00003,Viáticos f,x,232.000000,16/02/2023,17/02/2023,2,Culiacán,SIN,DG1,Dirección general
3,3,V00004,Viáticos f,x,2042.790039,01/03/2023,02/03/2023,2,Chihuahua,CHI,DG1,Dirección general
4,4,V00005,Viáticos f,x,116.000000,05/05/2023,05/05/2023,1,Reynosa,TMS,DG1,Dirección general
...,...,...,...,...,...,...,...,...,...,...,...,...
57313,57313,V13821,Gasolina rp,x,809.640015,18/08/2023,18/08/2023,1,Axochiapan,MOR,DG1,Estrategia
57314,57314,V12741,Viáticos rp,x,17370.000000,24/09/2023,07/10/2023,14,Morelia,MCH,DG1,Estrategia
57315,57315,V13855,Viáticos rp,x,53.790001,01/12/2023,01/12/2023,1,Tepalcingo,MOR,DG1,Estrategia
57316,57316,V13856,Viáticos rp,x,311.200012,08/12/2023,08/12/2023,1,Tepalcingo,MOR,DG1,Estrategia


In [5]:
viajes = viajes.loc[viajes['gasto_observable']=='x']
viajes

,Unnamed: 0,codigo_viaje,tipo_de_gasto,gasto_observable,monto,inicio,fin,Duración del viaje,destino_mpio,destino_edo,dirección,unidad
0,0,V00001,Viáticos f,x,190.000000,25/01/2023,26/01/2023,2,León,GTO,DG1,Dirección general
1,1,V00002,Viáticos f,x,171.000000,26/01/2023,27/01/2023,2,Morelia,MCH,DG1,Dirección general
2,2,V00003,Viáticos f,x,232.000000,16/02/2023,17/02/2023,2,Culiacán,SIN,DG1,Dirección general
3,3,V00004,Viáticos f,x,2042.790039,01/03/2023,02/03/2023,2,Chihuahua,CHI,DG1,Dirección general
4,4,V00005,Viáticos f,x,116.000000,05/05/2023,05/05/2023,1,Reynosa,TMS,DG1,Dirección general
...,...,...,...,...,...,...,...,...,...,...,...,...
57313,57313,V13821,Gasolina rp,x,809.640015,18/08/2023,18/08/2023,1,Axochiapan,MOR,DG1,Estrategia
57314,57314,V12741,Viáticos rp,x,17370.000000,24/09/2023,07/10/2023,14,Morelia,MCH,DG1,Estrategia
57315,57315,V13855,Viáticos rp,x,53.790001,01/12/2023,01/12/2023,1,Tepalcingo,MOR,DG1,Estrategia
57316,57316,V13856,Viáticos rp,x,311.200012,08/12/2023,08/12/2023,1,Tepalcingo,MOR,DG1,Estrategia


In [6]:
gasto_unidad=pd.DataFrame(viajes.groupby(['unidad'])['monto'].sum()).reset_index()
gasto_unidad['tipo_de_gasto']='Total'
gasto_unidad=gasto_unidad[['unidad','tipo_de_gasto','monto']]
gasto_unidad

,unidad,tipo_de_gasto,monto
0,Administración,Total,3.408728e+06
1,Atención corporativa,Total,7.005693e+05
2,Auditoría,Total,2.920409e+04
3,Auditoría interna,Total,4.640900e+05
4,Control,Total,1.086075e+05
5,Crédito,Total,3.541433e+05
6,Desarrollo y gestión,Total,3.482957e+04
7,Dirección administración,Total,1.617185e+04
8,Dirección de crédito,Total,1.078502e+05
9,Dirección de finanzas,Total,4.350357e+04


In [7]:
data1=pd.DataFrame(viajes.groupby(['unidad','tipo_de_gasto'])['monto'].sum()).reset_index()
data1

,unidad,tipo_de_gasto,monto
0,Administración,Gasolina f,32093.099883
1,Administración,Gasolina rp,363670.169649
2,Administración,Pasajes aéreos f,164693.570068
3,Administración,Pasajes aéreos rp,820410.090575
4,Administración,Pasajes terrestres f,48219.400052
...,...,...,...
229,Tesorería,Gasolina rp,3862.840058
230,Tesorería,Pasajes terrestres f,12033.579956
231,Tesorería,Pasajes terrestres rp,9337.380066
232,Tesorería,Viáticos f,51678.750517


In [8]:
data=pd.concat([data1,gasto_unidad]).reset_index()
data

,index,unidad,tipo_de_gasto,monto
0,0,Administración,Gasolina f,3.209310e+04
1,1,Administración,Gasolina rp,3.636702e+05
2,2,Administración,Pasajes aéreos f,1.646936e+05
3,3,Administración,Pasajes aéreos rp,8.204101e+05
4,4,Administración,Pasajes terrestres f,4.821940e+04
...,...,...,...,...
265,31,Riesgos,Total,5.343449e+04
266,32,Sistemas,Total,4.440407e+05
267,33,Sur,Total,6.892767e+06
268,34,Sureste,Total,3.870816e+06


In [9]:
direcciones = viajes[['unidad','dirección']]

In [10]:
direcciones['direccion_unidad']=viajes['unidad']+viajes['dirección']


C:\Users\Daniela\AppData\Local\Temp\ipykernel_26380\1823881552.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  direcciones['direccion_unidad']=viajes['unidad']+viajes['dirección']


In [11]:
direcciones_unique =direcciones.drop_duplicates(subset='direccion_unidad')
direcciones_unique.head()

,unidad,dirección,direccion_unidad
0,Dirección general,DG1,Dirección generalDG1
13,Programas y proyectos,DG5,Programas y proyectosDG5
14,Sistemas,DG6,SistemasDG6
15,Operaciones,DG6,OperacionesDG6
16,Administración,DG7,AdministraciónDG7


In [12]:
data = pd.merge(data,direcciones_unique[['unidad','dirección']],on='unidad',how='left')
data

,index,unidad,tipo_de_gasto,monto,dirección
0,0,Administración,Gasolina f,3.209310e+04,DG7
1,1,Administración,Gasolina rp,3.636702e+05,DG7
2,2,Administración,Pasajes aéreos f,1.646936e+05,DG7
3,3,Administración,Pasajes aéreos rp,8.204101e+05,DG7
4,4,Administración,Pasajes terrestres f,4.821940e+04,DG7
...,...,...,...,...,...
265,31,Riesgos,Total,5.343449e+04,DG1
266,32,Sistemas,Total,4.440407e+05,DG6
267,33,Sur,Total,6.892767e+06,DG9
268,34,Sureste,Total,3.870816e+06,DG9


In [13]:
data = data[['dirección', 'unidad', 'tipo_de_gasto', 'monto']]
data

,dirección,unidad,tipo_de_gasto,monto
0,DG7,Administración,Gasolina f,3.209310e+04
1,DG7,Administración,Gasolina rp,3.636702e+05
2,DG7,Administración,Pasajes aéreos f,1.646936e+05
3,DG7,Administración,Pasajes aéreos rp,8.204101e+05
4,DG7,Administración,Pasajes terrestres f,4.821940e+04
...,...,...,...,...
265,DG1,Riesgos,Total,5.343449e+04
266,DG6,Sistemas,Total,4.440407e+05
267,DG9,Sur,Total,6.892767e+06
268,DG9,Sureste,Total,3.870816e+06


In [35]:
data_sin_total =data.loc[data['tipo_de_gasto']!='Total']
totales=pd.DataFrame(data_sin_total.groupby(['tipo_de_gasto'])['monto'].sum()).reset_index()
totales['dirección']='Total'
totales['unidad']='Total'
totales.columns
totales=totales[['dirección', 'unidad','tipo_de_gasto', 'monto']]
totales

,dirección,unidad,tipo_de_gasto,monto
0,Total,Total,Gasolina f,5.243245e+05
1,Total,Total,Gasolina rp,7.281891e+06
2,Total,Total,Pasajes aéreos f,1.428837e+06
3,Total,Total,Pasajes aéreos rp,4.389166e+06
4,Total,Total,Pasajes terrestres f,4.482721e+05
5,Total,Total,Pasajes terrestres rp,2.975721e+06
6,Total,Total,Viáticos f,2.722788e+06
7,Total,Total,Viáticos rp,1.753991e+07


In [36]:
data=pd.concat([totales,data])
data

,dirección,unidad,tipo_de_gasto,monto
0,Total,Total,Gasolina f,5.243245e+05
1,Total,Total,Gasolina rp,7.281891e+06
2,Total,Total,Pasajes aéreos f,1.428837e+06
3,Total,Total,Pasajes aéreos rp,4.389166e+06
4,Total,Total,Pasajes terrestres f,4.482721e+05
...,...,...,...,...
265,DG1,Riesgos,Total,5.343449e+04
266,DG6,Sistemas,Total,4.440407e+05
267,DG9,Sur,Total,6.892767e+06
268,DG9,Sureste,Total,3.870816e+06


In [37]:
data_dict =data.to_dict(orient='records')

In [45]:
client =MongoClient('mongodb://localhost:27017/')
db = client['viajes']
collection = db['data']
collection.insert_many(data_dict)

In [46]:
client.close()

In [48]:
viajes

,Unnamed: 0,codigo_viaje,tipo_de_gasto,gasto_observable,monto,inicio,fin,Duración del viaje,destino_mpio,destino_edo,dirección,unidad
0,0,V00001,Viáticos f,x,190.000000,25/01/2023,26/01/2023,2,León,GTO,DG1,Dirección general
1,1,V00002,Viáticos f,x,171.000000,26/01/2023,27/01/2023,2,Morelia,MCH,DG1,Dirección general
2,2,V00003,Viáticos f,x,232.000000,16/02/2023,17/02/2023,2,Culiacán,SIN,DG1,Dirección general
3,3,V00004,Viáticos f,x,2042.790039,01/03/2023,02/03/2023,2,Chihuahua,CHI,DG1,Dirección general
4,4,V00005,Viáticos f,x,116.000000,05/05/2023,05/05/2023,1,Reynosa,TMS,DG1,Dirección general
...,...,...,...,...,...,...,...,...,...,...,...,...
57313,57313,V13821,Gasolina rp,x,809.640015,18/08/2023,18/08/2023,1,Axochiapan,MOR,DG1,Estrategia
57314,57314,V12741,Viáticos rp,x,17370.000000,24/09/2023,07/10/2023,14,Morelia,MCH,DG1,Estrategia
57315,57315,V13855,Viáticos rp,x,53.790001,01/12/2023,01/12/2023,1,Tepalcingo,MOR,DG1,Estrategia
57316,57316,V13856,Viáticos rp,x,311.200012,08/12/2023,08/12/2023,1,Tepalcingo,MOR,DG1,Estrategia


In [62]:
viajes['mes']=viajes['inicio'].str[3:5]
viajes['mes']=viajes['mes'].astype(int)
viajes['mes'].unique()

C:\Users\Daniela\AppData\Local\Temp\ipykernel_26380\2916365391.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  viajes['mes']=viajes['inicio'].str[3:5]
C:\Users\Daniela\AppData\Local\Temp\ipykernel_26380\2916365391.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  viajes['mes']=viajes['mes'].astype(int)


array([ 1,  2,  3,  5,  6,  7,  9, 10, 11, 12,  4,  8])

In [74]:
time_data = viajes.groupby(['unidad','tipo_de_gasto','mes'])['monto'].sum().reset_index()
time_data =time_data.sort_values(by=['unidad','tipo_de_gasto','mes'])

In [75]:
time_data['monto_acumulado'] = time_data.groupby(['unidad','tipo_de_gasto'])['monto'].cumsum()

In [76]:
time_data.loc[time_data['unidad']=='Tesorería']

,unidad,tipo_de_gasto,mes,monto,monto_acumulado
1213,Tesorería,Gasolina f,3,1629.040039,1629.040039
1214,Tesorería,Gasolina f,8,2082.229950,3711.269989
1215,Tesorería,Gasolina f,9,1789.680054,5500.950043
1216,Tesorería,Gasolina f,11,5684.179932,11185.129975
1217,Tesorería,Gasolina rp,1,1792.890015,1792.890015
1218,Tesorería,Gasolina rp,4,1068.810059,2861.700074
1219,Tesorería,Gasolina rp,11,1001.139984,3862.840058
1220,Tesorería,Pasajes terrestres f,1,959.119995,959.119995
1221,Tesorería,Pasajes terrestres f,3,3501.500000,4460.619995
1222,Tesorería,Pasajes terrestres f,8,1490.010010,5950.630005


In [77]:
time_data_dict =time_data.to_dict(orient='records')

In [78]:
client =MongoClient('mongodb://localhost:27017/')
db = client['viajes']
collection = db['time_data']
collection.insert_many(time_data_dict)

In [79]:
len(time_data)

1240